In [1]:
import pandas as pd
import pickle
country = {}
a_file = open("airport.pkl", "rb")
country = pickle.load(a_file)
a_file.close()
continents = {}
a_file = open("continents.pkl", "rb")
continents = pickle.load(a_file)
a_file.close()

In [2]:
board_pass = pd.read_csv('data/YourBoardingPassDotAero.csv', index_col='id')

In [3]:
board_pass.columns

Index(['Sequence', 'Title', 'Name', 'FlightNumber', 'BoardNumber', 'Gate',
       'From', 'FromCode', 'To', 'ToCode', 'Date', 'Time', 'Operated',
       'BoardingEnded', 'Seat', 'PNR', 'ETicket'],
      dtype='object')

### Берем только нужные столбцы для подсчета подозрительных полетов и через страны

In [4]:
board_pass = board_pass[['Name', 'FlightNumber', 'BoardNumber', 'From', 'FromCode', 'To', 'ToCode', 'Date', 'Time']]

In [5]:
board_pass

,Name,FlightNumber,BoardNumber,From,FromCode,To,ToCode,Date,Time
id,,,,,,,,,
0,LIDIYA ZHDANOVA,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25
1,SVIATOSLAV SOKOLOV,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25
2,SVIATOSLAV SUKHANOV,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25
3,DENIS ZELENIN,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25
4,OSTROVSKAYA IRINA,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25
...,...,...,...,...,...,...,...,...,...
615766,KORNILOV VLADIMIR T,AZ7240,NaN,MILAN,MXP,JEDDAH,JED,2018-01-01,11:00
615767,NELLI A SHESTAKOVA,DL1161,NaN,DAYTON OH,DAY,ATLANTA GA,ATL,2018-01-01,14:17
615768,KOLPAKOVA MARIYA,SU1502,NaN,MOSCOW,SVO,TYUMEN,TJM,2018-01-01,22:30


### Для перевода имени колонок к удобному виду

In [6]:
import re
def camel_to_snake(name):
  name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
  return re.sub('([a-z0-9])([A-Z])', r'\1_\2', name).lower()
new_cols = {i:camel_to_snake(i) for i in board_pass.columns}
board_pass.rename(columns=new_cols, inplace=True)

In [7]:
board_pass

,name,flight_number,board_number,from,from_code,to,to_code,date,time
id,,,,,,,,,
0,LIDIYA ZHDANOVA,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25
1,SVIATOSLAV SOKOLOV,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25
2,SVIATOSLAV SUKHANOV,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25
3,DENIS ZELENIN,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25
4,OSTROVSKAYA IRINA,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25
...,...,...,...,...,...,...,...,...,...
615766,KORNILOV VLADIMIR T,AZ7240,NaN,MILAN,MXP,JEDDAH,JED,2018-01-01,11:00
615767,NELLI A SHESTAKOVA,DL1161,NaN,DAYTON OH,DAY,ATLANTA GA,ATL,2018-01-01,14:17
615768,KOLPAKOVA MARIYA,SU1502,NaN,MOSCOW,SVO,TYUMEN,TJM,2018-01-01,22:30


### Если в графиках перелетов есть несостыковки из места прилета предыдущего и вылета текущего

In [8]:
def count_not_normal_flights(dataframe):
    prev = ''
    count = 0
    for elem in dataframe.values:
        if pd.isna(elem[0]):
            continue
        if pd.isna(elem[1]):
            prev = ''
            continue
        if prev != '' and elem[0].lower() != prev.lower():
            count += 1
        prev = elem[1]
    return count

### Перелет из одной страны в другую

In [9]:
def count_not_cross_country_flights(dataframe):
    prev = ''
    count = 0
    for elem in dataframe.values:
        if elem[0] == '' or elem[1] == '':
            continue
        if elem[0] != elem[1]:
            count += 1
    return count

### Определяем страны прилета и вылета по коду аэропорта

In [11]:
board_pass['dep_country'] = [country.get(elem,'') for elem in board_pass['from_code'].values]

In [12]:
board_pass['arr_country'] = [country.get(elem,'') for elem in board_pass['to_code'].values]

In [13]:
board_pass['dep_continent'] = [continents.get(elem,'') for elem in board_pass['dep_country'].values]

In [14]:
board_pass['arr_continent'] = [continents.get(elem,'') for elem in board_pass['arr_country'].values]

In [17]:
board_pass.loc[board_pass['to'] == 'SEOUL', 'arr_country'] = 'South Korea'

In [19]:
board_pass.loc[board_pass['from'] == 'SEOUL', 'dep_country'] = 'South Korea'

In [20]:
board_pass.loc[board_pass['to'] == 'SEOUL', 'arr_continent'] = 'Asia'

In [21]:
board_pass.loc[board_pass['from'] == 'SEOUL', 'arr_continent'] = 'Asia'

In [22]:
board_pass

,name,flight_number,board_number,from,from_code,to,to_code,date,time,dep_country,arr_country,dep_continent,arr_continent
id,,,,,,,,,,,,,
0,LIDIYA ZHDANOVA,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25,Russia,South Korea,Europe,Asia
1,SVIATOSLAV SOKOLOV,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25,Russia,South Korea,Europe,Asia
2,SVIATOSLAV SUKHANOV,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25,Russia,South Korea,Europe,Asia
3,DENIS ZELENIN,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25,Russia,South Korea,Europe,Asia
4,OSTROVSKAYA IRINA,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25,Russia,South Korea,Europe,Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...
615766,KORNILOV VLADIMIR T,AZ7240,NaN,MILAN,MXP,JEDDAH,JED,2018-01-01,11:00,Italy,Saudi Arabia,Europe,Asia
615767,NELLI A SHESTAKOVA,DL1161,NaN,DAYTON OH,DAY,ATLANTA GA,ATL,2018-01-01,14:17,USA,USA,North America,North America
615768,KOLPAKOVA MARIYA,SU1502,NaN,MOSCOW,SVO,TYUMEN,TJM,2018-01-01,22:30,Russia,Russia,Europe,Europe


In [48]:
board_pass

,name,flight_number,board_number,from,from_code,to,to_code,date,time,dep_country,arr_country,dep_continent,arr_continent
id,,,,,,,,,,,,,
0,LIDIYA ZHDANOVA,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25,Russia,South Korea,Europe,Asia
1,SVIATOSLAV SOKOLOV,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25,Russia,South Korea,Europe,Asia
2,SVIATOSLAV SUKHANOV,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25,Russia,South Korea,Europe,Asia
3,DENIS ZELENIN,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25,Russia,South Korea,Europe,Asia
4,OSTROVSKAYA IRINA,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25,Russia,South Korea,Europe,Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...
615766,KORNILOV VLADIMIR T,AZ7240,NaN,MILAN,MXP,JEDDAH,JED,2018-01-01,11:00,Italy,Saudi Arabia,Europe,Asia
615767,NELLI A SHESTAKOVA,DL1161,NaN,DAYTON OH,DAY,ATLANTA GA,ATL,2018-01-01,14:17,USA,USA,North America,North America
615768,KOLPAKOVA MARIYA,SU1502,NaN,MOSCOW,SVO,TYUMEN,TJM,2018-01-01,22:30,Russia,Russia,Europe,Europe


In [49]:
bp = pd.read_csv('data/Ready/not_norm_board_pass.csv', index_col='id')

In [51]:
result_bp = pd.merge(board_pass, bp, on='name', how='inner')

In [53]:
res_1 = result_bp[result_bp['flights'] > 1]

In [54]:
res_1

,name,flight_number,board_number,from,from_code,to,to_code,date,time,dep_country,arr_country,dep_continent,arr_continent,flights,not_ok_flights,cross_country,percent_not_ok,percent_cross
0,LIDIYA ZHDANOVA,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25,Russia,South Korea,Europe,Asia,4,3,0,0.75,0.0
1,LIDIYA ZHDANOVA,DL2229,NaN,ATLANTA GA,ATL,CHARLESTON SC,CHS,2017-03-24,10:46,USA,USA,North America,North America,4,3,0,0.75,0.0
2,LIDIYA ZHDANOVA,CZ6940,NaN,URUMQI,URC,WUHAN,WUH,2017-08-05,20:05,China,China,Asia,Asia,4,3,0,0.75,0.0
3,LIDIYA ZHDANOVA,VN4230,NaN,HO CHI MINH CITY,SGN,HANOI,HAN,2017-09-01,17:20,Vietnam,Vietnam,Asia,Asia,4,3,0,0.75,0.0
4,SVIATOSLAV SOKOLOV,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25,Russia,South Korea,Europe,Asia,5,4,2,0.80,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
615030,MESHCHERIAKOVA EVGENIIA,KL1977,KE 882063042,AMSTERDAM,AMS,BUDAPEST,BUD,2017-12-30,14:15,Netherlands,Hungary,Europe,Europe,2,0,2,0.00,1.0
615254,SHAROVA EVELINA,DL3973,NaN,BRISTOL/JOHNSON/KINGSPORT TN,TRI,ATLANTA GA,ATL,2017-12-31,09:59,USA,USA,North America,North America,2,1,0,0.50,0.0
615255,SHAROVA EVELINA,DL3342,SU 975195735,COLUMBUS MS,GTR,ATLANTA GA,ATL,2017-12-31,14:40,USA,USA,North America,North America,2,1,0,0.50,0.0
615433,KAROLINA KALACHEVA,CZ6951,NaN,URUMQI,URC,XINING,XNN,2017-12-31,08:10,China,China,Asia,Asia,2,0,0,0.00,0.0


In [56]:
res_1.drop(columns=['flights', 'not_ok_flights',
       'cross_country', 'percent_not_ok', 'percent_cross'], inplace=True)

D:\Programs\Anaconda\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [57]:
res_1

,name,flight_number,board_number,from,from_code,to,to_code,date,time,dep_country,arr_country,dep_continent,arr_continent
0,LIDIYA ZHDANOVA,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25,Russia,South Korea,Europe,Asia
1,LIDIYA ZHDANOVA,DL2229,NaN,ATLANTA GA,ATL,CHARLESTON SC,CHS,2017-03-24,10:46,USA,USA,North America,North America
2,LIDIYA ZHDANOVA,CZ6940,NaN,URUMQI,URC,WUHAN,WUH,2017-08-05,20:05,China,China,Asia,Asia
3,LIDIYA ZHDANOVA,VN4230,NaN,HO CHI MINH CITY,SGN,HANOI,HAN,2017-09-01,17:20,Vietnam,Vietnam,Asia,Asia
4,SVIATOSLAV SOKOLOV,SU5436,NaN,VLADIVOSTOK,VVO,SEOUL,REA,2017-01-01,11:25,Russia,South Korea,Europe,Asia
...,...,...,...,...,...,...,...,...,...,...,...,...,...
615030,MESHCHERIAKOVA EVGENIIA,KL1977,KE 882063042,AMSTERDAM,AMS,BUDAPEST,BUD,2017-12-30,14:15,Netherlands,Hungary,Europe,Europe
615254,SHAROVA EVELINA,DL3973,NaN,BRISTOL/JOHNSON/KINGSPORT TN,TRI,ATLANTA GA,ATL,2017-12-31,09:59,USA,USA,North America,North America
615255,SHAROVA EVELINA,DL3342,SU 975195735,COLUMBUS MS,GTR,ATLANTA GA,ATL,2017-12-31,14:40,USA,USA,North America,North America
615433,KAROLINA KALACHEVA,CZ6951,NaN,URUMQI,URC,XINING,XNN,2017-12-31,08:10,China,China,Asia,Asia


In [58]:
board_pass = res_1.copy()

### Создаем таблицу для записи данных

In [64]:
not_normal_flights = pd.DataFrame(columns=['name', 'flights', 'not_ok_flights', 'cross_country', 'cross_country_not_ok', 'long_flights'])

In [65]:
len(board_pass.name.unique())

127532

## Считаем все: not_ok, между народные, ненормальные международные и кроссконтинентальные

In [66]:
def count_not_normal_flights(dataframe):
    prev_air = ''
    not_ok = 0
    cross_country = 0
    cross_not_ok = 0
    long = 0
    for elem in dataframe.values:
        from_air = elem[0]
        to_air = elem[1]
        from_coun = elem[2]
        to_coun = elem[3]
        from_con = elem[4]
        to_con = elem[5]
        if pd.isna(from_air):
            continue
        if pd.isna(to_air):
            prev_air = ''
            continue
        if prev_air != '' and from_air.lower() != prev_air.lower():
            not_ok += 1
            if from_coun != '' and to_coun != '':
                if from_coun.lower() != to_coun.lower():
                    cross_not_ok += 1
        prev_air = to_air
        if from_coun != '' and to_coun != '':
            if from_coun.lower() != to_coun.lower():
                cross_country += 1
        if from_con != '' and to_con != '':
            if from_con.lower() != to_con.lower():
                long += 1
    return not_ok, cross_country, cross_not_ok, long

In [67]:
value = 5000
unique_names = board_pass.name.unique()
for i in range (len(unique_names)):
    name = unique_names[i]
    sub_df = board_pass[board_pass['name'] == name].sort_values(by='date').reset_index(drop=True)
    sub_df_1 = sub_df[['from_code', 'to_code', 'dep_country', 'arr_country', 'dep_continent', 'arr_continent']]
    count_flights = len(sub_df)
    not_ok, cross_country, cross_not_ok, long = count_not_normal_flights(sub_df_1)
    not_normal_flights.loc[i] = [name] + [count_flights] + [not_ok] + [cross_country] + [cross_not_ok] + [long]
    if i == value:
        print(i)
        value += 5000

5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000
80000
85000
90000
95000
100000
105000
110000
115000
120000
125000


In [68]:
not_normal_flights

,name,flights,not_ok_flights,cross_country,cross_country_not_ok,long_flights
0,LIDIYA ZHDANOVA,4,3,1,0,1
1,SVIATOSLAV SOKOLOV,5,4,3,2,2
2,SVIATOSLAV SUKHANOV,8,4,4,2,3
3,DENIS ZELENIN,7,2,1,0,1
4,OSTROVSKAYA IRINA,4,2,2,0,1
...,...,...,...,...,...,...
127527,DANILA KOPYLOV,2,0,2,0,0
127528,ALLA KOROL'KOVA,2,1,2,1,2
127529,MESHCHERIAKOVA EVGENIIA,2,0,2,0,0
127530,SHAROVA EVELINA,2,1,0,0,0


### Вычисляем процент подозрительных рейсов и процент международных рейсов

In [69]:
not_normal_flights['percent_not_ok'] = [elem[2] / elem[1] for elem in not_normal_flights.values]

In [70]:
not_normal_flights['percent_cross'] = [elem[3] / elem[1] for elem in not_normal_flights.values]

In [71]:
not_normal_flights.to_csv('data/Ready/not_norm_board_pass_1.csv', index_label='id')